In [ ]:
# %load "firstcell.py"
import numpy
import matplotlib
from matplotlib import pylab, mlab, pyplot
np = numpy
plt = pyplot

from IPython.core.pylabtools import figsize, getfigs

from pylab import *
from numpy import *

In [ ]:
from cycler import cycler

In [ ]:
import pathlib
import json
import bitstruct
import crcmod
import scipy.stats
from tqdm.notebook import trange, tqdm

from ECOND_SEU import run_times, fluences, hexa42_files, hexa43_files, hexprint, yerr, pois_err, items, packet_parser, capture_batch, incident, run
from ECOND_SEU import PRBS_run_times, PRBS_fluences, hexa42_PRBS_files, hexa43_PRBS_files

In [ ]:
hexa_files_location = '/home/jsw/tmp/ECOND_Jan2024_SEU_logs/logs/'

In [ ]:
%%time
J43 = []
percaps = []
for i, fname in enumerate(hexa43_files):
    if i < 9:
        percaps.append(8)
    else:
        percaps.append(5)
    with open(pathlib.Path(hexa_files_location, fname)) as jsonfile:
        J43.append(json.load(jsonfile))

In [ ]:
%%time
J42 = []
percaps = []
for i, fname in enumerate(hexa42_files):
    if i < 9:
        percaps.append(8)
    else:
        percaps.append(5)
    with open(pathlib.Path(hexa_files_location, fname)) as jsonfile:
        J42.append(json.load(jsonfile))

In [ ]:
%%time
prebeam = packet_parser(J42[0])

In [ ]:
R42 = [run(J42[k], percaps[k], prebeam) for k in trange(16)]

In [ ]:
R43 = [run(J43[k], percaps[k], prebeam) for k in trange(16)]

In [ ]:
with np.load("all58.npz") as f:
    patt5 = f['all5']
    patt8 = f['all8']
    e_bits = f['e_bits']
    e_words = f['e_words']
    e_BXs = f['e_BXs']
    e_specs = f['e_specs']

In [ ]:
with np.load('false_pileup.npz') as f:
    double_patt8 = f['double_patt8']
    double_i8 = f['double_i8']
    double_j8 = f['double_j8']
    total_double8 = f['total8']
    double_patt5 = f['double_patt5']
    double_i5 = f['double_i5']
    double_j5 = f['double_j5']
    total_double5 = f['total5']

In [ ]:
with np.load('OB_double_SEU.npz') as f:
    OB_BX_offset_patt8 = f['OB_BX_offset_patt8']
    OB_bit_offset_patt8 = f['OB_bit_offset_patt8']
    OB_bit_offset_valid8 = f['OB_bit_offset_valid8']
    OB_bit_offset_patt5 = f['OB_bit_offset_patt5']
    OB_bit_offset_valid5 = f['OB_bit_offset_valid5']

In [ ]:
# MC_patt5 = unique(concatenate([patt5, double_patt5, OB_bit_offset_patt5[OB_bit_offset_valid5]]))
# MC_patt8 = unique(concatenate([patt8, double_patt8, OB_bit_offset_patt8[OB_bit_offset_valid8], OB_BX_offset_patt8]))
MC_patt5 = unique(concatenate([patt5, double_patt5]))
MC_patt8 = unique(concatenate([patt8, double_patt8]))

len(MC_patt5), len(MC_patt8)

In [ ]:
def calc_PP_indices():
    # These four lines come directly from test_proton_seu.py
    np.random.seed(15665843)
    L1A_count=67
    l1a_bx=np.random.choice(np.arange(10,3450),L1A_count,replace=False)
    l1a_bx.sort()

    # Now we need to offset the start time enough to finish preceding
    # packets, and further offset it by 54 BX to make it match up with
    # the error data.
    prev_l = -inf
    ls = []
    for l in l1a_bx:
        newl = max(l, prev_l + 41)
        ls.append(newl+54)
        prev_l = newl
    ls = array(ls)
    ls[52] += 1 # Not sure why this is necessary, but it makes the analysis line up properly.

    # And then we need to find the distance to the immediately
    # preceding PP readout start for every BX, wrapping around
    # the end of the orbit as needed.
    return ((arange(3564) - ls[(searchsorted(ls, arange(3564), side='right') - 1) % len(ls)]) % 3564)
PP_indices = calc_PP_indices()

def PP_reads():
    # Start from 0, -4 to accommodate the packet header
    x = 0
    y = -4
    for i in range(472):
        yield x, y, i//12
        if y >= 0:
            x += 1
        y += 1
        if x == 12:
            x = 0
        if y == 37:
            # Go to -2 to accommodate the subpacket headers
            y = -2
    # Note that the last two (of 472) have negative y. This is right
    # because this is the packet trailer.

# This is the amount to add to the difference between the write and read times
PP_baseline_latency = 40
    
# Make this big enough to accommodate the largest of PP_indices
PP_dwell = zeros((12,300), int)
for x, y, BX in PP_reads():
    # BX is the read out time
    # x is the RAM (and therefore the error word)
    # y tells us the read in time
    # if x or y is negative, then it's not really in the RAM so we ignore it
    if (x >= 0) and (y >= 0):
        PP_dwell[11-x, BX] = BX - y + PP_baseline_latency
assert PP_dwell.min() >= 0

In [ ]:
dat = []
with open('/home/jsw/src/econd-sw/RAM_rw') as f:
    for line in f:
        l = line.strip()
        if len(l) > 0:
            RAstr, WAstr, REstr, WEstr, RDstr, WDstr = l.split(',')
            if 'x' in WAstr: continue
            RAs = [int(RAstr[2*i:2*(i+1)], 16) for i in range(12)]
            WAs = [int(WAstr[2*i:2*(i+1)], 16) for i in range(12)]
            REs = [(int(REstr,16) >> (11 - i)) & 0x1 for i in range(12)]
            WEs = [int(WEstr) for i in range(12)]
            RDs = [int(RDstr[8*i:8*(i+1)], 16) if 'x' not in RDstr[8*i:8*(i+1)] else -1 for i in range(12)]
            WDs = [int(WDstr[8*i:8*(i+1)], 16) if 'x' not in WDstr[8*i:8*(i+1)] else -1 for i in range(12)]
            dat.append([RAs, WAs, REs, WEs, RDs, WDs])
            
dat = array(dat)

RAMt = zeros((12, 256), uint)
RAM = zeros((12, 256), uint32)

dwells = []
j = 0
q = 0
prev_dwell = 45
for i in range(dat.shape[0]-1):
    RAs, WAs, REs, WEs, RDs, WDs = dat[i]
    for k in range(12):
        if WEs[k] == 0:
            RAMt[k,WAs[k]] = i
            RAM[k, WAs[k]] = WDs[k]
    if any(REs == 0):
        while True:
            if REs[j] == 0:
                dwellt = uint64(i) - RAMt[j, RAs[j]]
                if dwellt < 10 and prev_dwell > 10:
                    # Start of new packet
                    q = 0
                else:
                    q = (q + 1) % 6
                prev_dwell = dwellt
                dwells.append((dwellt, RAMt[j,RAs[j]], 
                               i, j, RAs[j], q, RAM[j,RAs[j]], dat[i+1,4,j]))
                j = (j + 1) % 12
            else:
                break
dwells = array(dwells, int64)

# dwell_splits = (diff(1*(dwells[:,0] < 10)) == 1).nonzero()[0]

assert all(repeat(roll(dwells, 278, axis=0)[:,5], 32) == e_words[(e_specs == 2)])

assert all(repeat((roll(dwells, 278, axis=0)[:,2] + 57)%3564, 32) == e_BXs[(e_specs == 2)])

OB_dwell = zeros_like(e_specs)
OB_dwell[e_specs == 2] = repeat(roll(dwells, 278, axis=0)[:,0], 32)

In [ ]:
# In the eRX, the MSB (bit 31) comes in first, so it has the longest dwell
# time in untriplicated registers.  Each subsequent bit should have a dwell
# time that is 1 more cycle of a 1280 MHz clock.  There may be some baseline
# offset, such that bit 0 has a dwell time of a few cycles of the 1280 MHz
# clock, so we'll include that here as well, with an initial guess of 4.
spec0_deser_baseline = 31
spec0_align_baseline = 4
spec0_FF_dwelltime = ((e_bits+1)//2 + spec0_deser_baseline + spec0_align_baseline) * 25e-9 / 32
spec0_LATCH_dwelltime = (e_bits // 2) * 25e-9 / 32

# These values are taken from the tagged code that we used on Jan 27:
# https://gitlab.cern.ch/hgcal-daq-sw/econd-sw/-/blob/SEU_test_Jan2024_v2/test_radiation/conftest.py#L247-250
phase_select_42 = array([ 5,  5,  5,  5,  6,  7,  7,  8,  8,  2,  2,  2], uint8)
phase_select_43 = array([ 5,  12,  5, 12,  6,  7,  7,  8,  7, 10,  2,  2], uint8)
# Phase select has 15 possible values for a DLL
# The total length of the DLL is two entire bit intervals, or 2 * 25e-9 / 32 nanoseconds
# So, the dwell time in the delay line in nanoseconds is phase_select * (2 * 25e-9 / 32) / 15
spec0_DLL_dwelltime42 = (phase_select_42[e_words] + 0.5) * (2 * 25e-9 / 32) / 15
spec0_DLL_dwelltime43 = (phase_select_43[e_words] + 0.5) * (2 * 25e-9 / 32) / 15

# In the Ping-Pong RAM, the dwell time is always in an integer number of
# 40 MHz clock cycles.
spec1_dwelltime = PP_dwell[11-e_words, PP_indices[e_BXs]] * 25e-9

# We calculated the dwell time using simulation of the ECON-D ASIC
spec2_dwelltime = OB_dwell * 25e-9

# In the serializer, bit 31 (the MSB) lives in a 32-bit register clocked at
# 40 MHz for 1 cycle of a 640 MHz clock, then it lives in a 2-bit register
# clocked at 640 MHz for 1 cycle of a 1280 MHz clock, then it lives in a 1-bit
# register clocked at 1280 MHz for 1 cycle of a 1280 MHz clock.  So it lives
# for a total of 4 cycles of the 1280 MHz clock in untriplicated registers.
# Bit 30 lives for 5 cycles of the 1280 MHz clock, bit 29 for 6 cycles, etc.,
# down to bit 0 which lives for 35 cycles of the 1280 MHz clock in
# untriplicated registers.
spec3_baseline = 4
spec3_dwelltime = ((31 - e_bits) + spec3_baseline) * 25e-9 / 32

In [ ]:
N_sigma = 4
dwelltime = zeros((N_sigma, e_specs.shape[0]), dtype=float)
dwelltime[0] = spec0_LATCH_dwelltime * (e_specs == 0)
dwelltime[0] += spec0_FF_dwelltime * (e_specs == 0)
dwelltime[1] = spec1_dwelltime * (e_specs == 1)
dwelltime[2] = spec2_dwelltime * (e_specs == 2)
dwelltime[3] = spec3_dwelltime * (e_specs == 3)

dwelltime42 = copy(dwelltime)
dwelltime42[0] += spec0_DLL_dwelltime42 * (e_specs == 0)

dwelltime43 = copy(dwelltime)
dwelltime43[0] += spec0_DLL_dwelltime43 * (e_specs == 0)



# Convert dwelltime to fraction of an orbit
dwelltime42 /= (3564 * 25e-9)
dwelltime43 /= (3564 * 25e-9)

In [ ]:
D5_42 = zeros((N_sigma + N_sigma**2, MC_patt5.shape[0]), float)
key = searchsorted(MC_patt5, patt5)
for j in trange(patt5.shape[0]):
    D5_42[:N_sigma,key[j]] += dwelltime42[:,j]
double_key = searchsorted(MC_patt5, double_patt5)
for k in trange(double_patt5.shape[0]):
    D5_42[N_sigma:,double_key[k]] += (dwelltime42[:,newaxis,double_i5[k]] * dwelltime42[newaxis,:,double_j5[k]]).flatten() * sum(total_double5) / len(double_patt5)

In [ ]:
D8_42 = zeros((N_sigma + N_sigma**2, MC_patt8.shape[0]), float)
key = searchsorted(MC_patt8, patt8)
for j in trange(patt8.shape[0]):
    D8_42[:N_sigma,key[j]] += dwelltime42[:,j]
double_key = searchsorted(MC_patt8, double_patt8)
for k in trange(double_patt8.shape[0]):
    D8_42[N_sigma:,double_key[k]] += (dwelltime42[:,newaxis,double_i8[k]] * dwelltime42[newaxis,:,double_j8[k]]).flatten() * sum(total_double8) / len(double_patt8)

In [ ]:
D5_43 = zeros((N_sigma + N_sigma**2, MC_patt5.shape[0]), float)
key = searchsorted(MC_patt5, patt5)
for j in trange(patt5.shape[0]):
    D5_43[:N_sigma,key[j]] += dwelltime43[:,j]
double_key = searchsorted(MC_patt5, double_patt5)
for k in trange(double_patt5.shape[0]):
    D5_43[N_sigma:,double_key[k]] += (dwelltime43[:,newaxis,double_i5[k]] * dwelltime43[newaxis,:,double_j5[k]]).flatten() * sum(total_double5) / len(double_patt5)

In [ ]:
D8_43 = zeros((N_sigma + N_sigma**2, MC_patt8.shape[0]), float)
key = searchsorted(MC_patt8, patt8)
for j in trange(patt8.shape[0]):
    D8_43[:N_sigma,key[j]] += dwelltime43[:,j]
double_key = searchsorted(MC_patt8, double_patt8)
for k in trange(double_patt8.shape[0]):
    D8_43[N_sigma:,double_key[k]] += (dwelltime43[:,newaxis,double_i8[k]] * dwelltime43[newaxis,:,double_j8[k]]).flatten() * sum(total_double8) / len(double_patt8)

In [ ]:
def make_merges(D, Nbins = 200):
    M = eye(D.shape[1])
    max_acos = 2
    Dnew = copy(D[:N_sigma+1])
    Dnew[N_sigma] = D[N_sigma:].sum(axis=0)
    while M.shape[0] > Nbins:
        d = (asmatrix(M) * asmatrix(Dnew).T).A
        d /= sqrt((d**2).sum(axis=1, keepdims=True))
        A, B = triu_indices(d.shape[0], k=1)
        cos_angle = sum(d[A] * d[B], axis=1) #/ sqrt(sum(d[A]**2, axis=1) * sum(d[B]**2, axis=1))
        n = argmax(cos_angle)
        max_cos_angle = cos_angle[n]
        M[A[n]] += M[B[n]]
        M = delete(M, B[n], axis=0)
    return M

In [ ]:
M8 = make_merges(D8_42 + D8_43, 25)
M5 = make_merges(D5_42 + D5_43, 25)

In [ ]:
import torch
def analysis(R, plot_title, D, MC_patt, M, fluence, do_plot=False, fudge=1):
    Ddata = zeros(MC_patt.shape[0], int32)
    for i, p in enumerate(MC_patt):
        Ddata[i] += sum(R.incidents.pattern == p)
    
    # N_sigma = 4
    assert D.shape[0] == N_sigma * (N_sigma + 1)
    
    D = (asmatrix(M) * asmatrix(D).T).A.T * fluence
    D1 = D[:N_sigma].T
    D2 = D[N_sigma:].T.reshape(-1,N_sigma,N_sigma)
    
    Ddata = (asmatrix(M) * asmatrix(Ddata).T).A.flatten()
    
    tD1 = torch.tensor(D1, requires_grad=False)
    tD2 = torch.tensor(D2, requires_grad=False)
    tDdata = torch.tensor(Ddata, requires_grad=False)
    
    def tlambda(tbeta):
        return ((tD1 * tbeta).sum(axis=1) +
                fudge*fluence * (tD2 * torch.outer(tbeta, tbeta)).sum(axis=(1,2)) / (tD1 * tbeta).sum())
    
    def tlogpdf(tbeta):
        return -torch.distributions.Poisson(tlambda(tbeta)).log_prob(tDdata).sum()

    def logpdf(beta):
        tbeta = torch.tensor(abs(beta))
        return tlogpdf(tbeta).detach().numpy().item()

    def jac(beta):
        tbeta = torch.tensor(abs(beta), requires_grad=True)
        lp = tlogpdf(tbeta)
        lp.backward()
        return tbeta.grad.detach().numpy()

    def hesse(beta):
        tbeta = torch.tensor(abs(beta), requires_grad=True)
        return torch.autograd.functional.hessian(tlogpdf, tbeta).detach().numpy()
    
    res = scipy.optimize.minimize(logpdf, exp(randn(N_sigma)*0.7) * ones(N_sigma) * sum(Ddata) / sum(D1),
                                  method='Nelder-Mead', options={'fatol': 1e-7, 'xatol':1e-19, 'maxiter': 5000, 'maxfev': 5000, 'disp':False})
    # res = scipy.optimize.minimize(logpdf, res.x,
    #                               jac=jac, hess=hesse, method='Newton-CG', options={'xtol': 1e-15, 'disp':True})
    # print(res)
    # print()
    beta = abs(res.x)
    H = hesse(beta)
    Cov = linalg.inv(H)
    # unc = sqrt(diag(asmatrix(D).T * asmatrix(Cov) * asmatrix(D)))
    
    tbeta = torch.tensor(abs(beta), requires_grad=False)
    lamb = tlambda(tbeta)
    # inds = argsort(lamb.detach().numpy())[::-1]
    inds = argsort(sum(D1 * beta, axis=1) + sum(D2 * outer(beta, beta), axis=(1,2)) * fluence / sum(D1 * beta))[::-1]
    # keep = ((Ddata > 0) | ((D*beta[:,newaxis]).sum(axis=0) > 0.00))
    # inds = inds[keep[inds]]
    
    # figure(figsize=(3.375, 3.375*7/8), dpi=350)
    if do_plot:
        # fig = figure(figsize=(8,4), dpi=150, layout="constrained")
        # sfigs = fig.subfigures(2, 1, height_ratios=[3, 1])
        fig, axes = subplots(2, 1, sharex=True, figsize=(8,4), dpi=150, layout="constrained", gridspec_kw={'height_ratios': [3,1], 'hspace': 0})
        sca(axes[0])
        # axes[0].set_prop_cycle(cycler('color', ['#006BA4', '#5F9ED1', '#FF800E', '#BBBBBB', '#696969', '#00ff00', '#898989', '#A2C8EC', '#FFBC79', '#CFCFCF']))
        axes[0].set_prop_cycle(cycler('color', ['#006BA4', '#FF800E', '#BBBBBB', '#696969', '#00ff00', '#898989', '#A2C8EC', '#FFBC79', '#CFCFCF']))
        bin_edges = arange(inds.shape[0]+1) - 0.5
        DD = concatenate([zeros_like(lamb)[newaxis,:],
                          cumsum(D1 * beta, axis=1).T,
                          # cumsum([tlambda(tbeta * torch.eye(N_sigma)[i]).detach().numpy() for i in range(N_sigma)], axis=0),
                          lamb.detach().numpy()[newaxis,:]], axis=0)

        names = ['eRX', 'PP', 'OB', 'eTX', 'fake pileup']
        for i in range(N_sigma+1)[::1]:
            fill_between(repeat(bin_edges,2)[1:-1], repeat(DD[i][inds], 2), repeat(DD[i+1][inds], 2), label=names[i], lw=0)

        legend(loc='upper right', ncol=2)

        errorbar(x=arange(inds.shape[0]), y=Ddata[inds], yerr=yerr(Ddata[inds]) * (Ddata[inds] != 0),
                 ls='none', color='k', marker='.', markersize=3, lw=0.6)

        xlim(-0.5, inds.shape[0]-0.5)
        ylim(1e-2, 5e3)
        yscale('log')
        grid()
        title(plot_title)
        ylabel('Incidents')
        
        sca(axes[1])
        errorbar(x=arange(inds.shape[0])[Ddata[inds] != 0], y=(Ddata[inds] / DD[-1][inds])[Ddata[inds] != 0], yerr=(yerr(Ddata[inds]) / DD[-1][inds])[:,Ddata[inds] != 0],
                 ls='none', color='k', marker='.', markersize=3, lw=0.6)
        ylim(0, 3)
        yticks([0,1,2,3])
        grid()
        ylabel('Data / pred.')
        xlabel('Incident category (arbitrary order)')
        
        # tight_layout()
        # show()
        if isinstance(do_plot, str):
            savefig(f'{do_plot}.png')
            savefig(f'{do_plot}.pdf')
        show()
        close()
    
    return beta, Cov, H, inds, Ddata, D1, D2

In [ ]:
ret = [analysis(sum(R43[:9]), "Jan 27, 2024 SEU data: Hexa 43, Runs 1-9",   D8_43, MC_patt8, M8, sum(fluences[:9]), do_plot='H43_8'),
       analysis(sum(R43[9:]), "Jan 27, 2024 SEU data: Hexa 43, Runs 10-16", D5_43, MC_patt5, M5, sum(fluences[9:]), do_plot='H43_5'),
       analysis(sum(R42[:9]), "Jan 27, 2024 SEU data: Hexa 42, Runs 1-9",   D8_42, MC_patt8, M8, sum(fluences[:9]), do_plot='H42_8'),
       analysis(sum(R42[9:]), "Jan 27, 2024 SEU data: Hexa 42, Runs 10-16", D5_42, MC_patt5, M5, sum(fluences[9:]), do_plot='H42_5'),
      ]

In [ ]:
hexas = [43, 43, 42, 42]
runs = ['1-9', '10-16', '1-9', '10-16']
names = ['eRX', 'PP', 'OB', 'eTX']
for j, (beta, C, H, inds, Ddata, D1, D2) in enumerate(ret):
    print(f'Hexa{hexas[j]}, Runs {runs[j]}')
    print('='*60)
    for i in range(N_sigma):
        print(f'\t{names[i]:>12s} σ = ({beta[i]*1e15:7.2f} ± {sqrt(C[i,i])*1e15:7.2f})×10⁻¹⁵ cm²')
    print()
    print('\tCorrelations:')
    corr = C / sqrt(outer(diag(C), diag(C)))
    for i in range(N_sigma):
        print(f'\t\t[' + '   '.join([f'{corr[i,j]*100:6.2f}' for j in range(N_sigma)]) + ']')
    print()
    print()

In [ ]:
%%time
PRBS_J42 = []
percaps = []
for i, fname in enumerate(hexa42_PRBS_files):
    with open(pathlib.Path(hexa_files_location, fname)) as jsonfile:
        PRBS_J42.append(json.load(jsonfile))

PRBS_J43 = []
percaps = []
for i, fname in enumerate(hexa43_PRBS_files):
    with open(pathlib.Path(hexa_files_location, fname)) as jsonfile:
        PRBS_J43.append(json.load(jsonfile))

In [ ]:
class run_data:
    def __init__(self, R, D, MC_patt, M, fluence):
        self.R = R
        self.D = D
        self.MC_patt = MC_patt
        self.M = M
        self.fluence = fluence
        
        self.Ddata = zeros(MC_patt.shape[0], int32)
        for i, p in enumerate(MC_patt):
            self.Ddata[i] += sum(R.incidents.pattern == p)
        self.MDdata = (asmatrix(M) * asmatrix(self.Ddata).T).A.flatten()
            
        self.N_sigma = int((sqrt(1 + 4*D.shape[0]) - 1) / 2)
        assert self.N_sigma**2 + self.N_sigma == D.shape[0]
        
        self.MD = (asmatrix(M) * asmatrix(self.D).T).A.T * fluence
        self.D1 = self.MD[:self.N_sigma].T
        self.D2 = self.MD[self.N_sigma:].T.reshape(-1, self.N_sigma, self.N_sigma)
        
        self.tD1 = torch.tensor(self.D1, requires_grad=False)
        self.tD2 = torch.tensor(self.D2, requires_grad=False)
        self.tDdata = torch.tensor(self.MDdata, requires_grad=False)
        
    def tlambda(self, tbeta):
        return ((self.tD1 * tbeta).sum(axis=1) +
                self.fluence * (self.tD2 * torch.outer(tbeta, tbeta)).sum(axis=(1,2)) / (self.tD1 * tbeta).sum())
    
    def tlogpdf(self, tbeta):
        return -torch.distributions.Poisson(self.tlambda(tbeta)).log_prob(self.tDdata).sum()

In [ ]:
class PRBS_run_data:
    def __init__(self, Js, fluence, dwelltime, e_bits, e_words, e_specs):
        self.N_sigma = 4
        self.ASIC = concatenate([array(J['tests'][2]['metadata']['daq_asic_post_beam'], uint32) for J in Js])
        self.xor = (self.ASIC ^ numpy.roll(self.ASIC, 1, axis=1)) & (self.ASIC ^ numpy.roll(self.ASIC, -1, axis=1))
        self.fluence = fluence
        
        self.row, self.eTX = self.xor.nonzero()
        
        self.bit_index = asarray(ceil(log2(self.xor[self.row, self.eTX])), int)
        self.err_per_bit = bincount(self.bit_index, minlength=32)

        self.terr_per_bit = torch.tensor(self.err_per_bit, requires_grad=False)
        
        self.PRBS15_errs = sum([J['tests'][2]['metadata']['prbs_15_err_count'][-1][1] for J in Js], axis=0)
        self.tPRBS15_errs = torch.tensor(self.PRBS15_errs, requires_grad=False)
        
        self.dwelltime = dwelltime
        
        self.dwell7 = array([sum(self.dwelltime[:,(e_bits == i) & (e_specs == 3)], axis=1) for i in range(32)]) * self.fluence
        self.tdwell7 = torch.tensor(self.dwell7, requires_grad=False)
        
        self.dwell15 = array([sum(self.dwelltime[:,(e_words == i) & (e_specs == 0)], axis=1) for i in range(12)]) * self.fluence
        self.tdwell15 = torch.tensor(self.dwell15, requires_grad=False)

    def tlogpdf(self, tbeta):
        trate = self.fluence * (torch.arange(31, -1, -1) + 4) * 6 / 32
        return -(torch.distributions.Poisson((self.tdwell7 * tbeta).sum(axis=1)).log_prob(self.terr_per_bit).sum() +
                 torch.distributions.Poisson((self.tdwell15 * tbeta).sum(axis=1)).log_prob(self.tPRBS15_errs).sum())

In [ ]:
PRBS_42 = PRBS_run_data(PRBS_J42, sum(PRBS_fluences), dwelltime42, e_bits, e_words, e_specs)
PRBS_43 = PRBS_run_data(PRBS_J43, sum(PRBS_fluences), dwelltime43, e_bits, e_words, e_specs)

R8_43 = run_data(sum(R43[:9]), D8_43, MC_patt8, M8, sum(fluences[:9]))
R5_43 = run_data(sum(R43[9:]), D5_43, MC_patt5, M5, sum(fluences[9:]))
R8_42 = run_data(sum(R42[:9]), D8_42, MC_patt8, M8, sum(fluences[:9]))
R5_42 = run_data(sum(R42[9:]), D5_42, MC_patt5, M5, sum(fluences[9:]))

In [ ]:
import torch
def analysis(Rs):
    def tlogpdf(tbeta):
        return torch.stack([R.tlogpdf(tbeta) for R in Rs], dim=0).sum()
    
    def logpdf(beta):
        tbeta = torch.tensor(abs(beta))
        return tlogpdf(tbeta).detach().numpy().item()

    def hesse(beta):
        tbeta = torch.tensor(abs(beta), requires_grad=True)
        return torch.autograd.functional.hessian(tlogpdf, tbeta).detach().numpy()
    
    res = scipy.optimize.minimize(logpdf, exp(randn(N_sigma)*0.7) * 1.5e-14,
                                  method='Nelder-Mead', options={'fatol': 1e-7, 'xatol':1e-19, 'maxiter': 5000, 'maxfev': 5000, 'disp':False})

    # print(res)
    # print()
    beta = abs(res.x)
    H = hesse(beta)
    return beta, H

In [ ]:
sqrt(diag(scipy.linalg.inv(H[[0,3]][:,[0,3]])))

In [ ]:
ret_comb = [analysis([R8_42, R5_42, PRBS_42]),
        analysis([R8_43, R5_43, PRBS_43])]

In [ ]:
hexas = [42, 43]
runs = ['1-16 + PRBS (Runs 17-19 and 22)']*2
names = ['eRX', 'PP', 'OB', 'eTX']
for j, (beta, H) in enumerate(ret_comb):
    C = scipy.linalg.inv(H)
    print(f'Hexa{hexas[j]}, Runs {runs[j]}')
    print('='*60)
    for i in range(N_sigma):
        print(f'\t{names[i]:>12s} σ = ({beta[i]*1e15:7.2f} ± {sqrt(C[i,i])*1e15:7.2f})×10⁻¹⁵ cm²')
    print()
    print('\tCorrelations:')
    corr = C / sqrt(outer(diag(C), diag(C)))
    for i in range(N_sigma):
        print(f'\t\t[' + '   '.join([f'{corr[i,j]*100:6.2f}' for j in range(N_sigma)]) + ']')
    print()
    print()

In [ ]:
ret_PRBS = [analysis([PRBS_42]),
            analysis([PRBS_43])]

In [ ]:
hexas = [42, 43]
runs = ['PRBS (Runs 17-19 and 22)']*2
names = ['eRX', 'eTX']
indices = [0, 3]
for j, (beta, H) in enumerate(ret_PRBS):
    C = scipy.linalg.inv(H[indices][:,indices])
    beta = beta[indices]
    print(f'Hexa{hexas[j]}, Runs {runs[j]}')
    print('='*60)
    for i in range(len(indices)):
        print(f'\t{names[i]:>12s} σ = ({beta[i]*1e15:7.2f} ± {sqrt(C[i,i])*1e15:7.2f})×10⁻¹⁵ cm²')
    print()
    print('\tCorrelations:')
    corr = C / sqrt(outer(diag(C), diag(C)))
    for i in range(len(indices)):
        print(f'\t\t[' + '   '.join([f'{corr[i,j]*100:6.2f}' for j in range(len(indices))]) + ']')
    print()
    print()